## MNST image download

In [1]:
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
from tqdm import tqdm

In [2]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)

In [3]:
len(dataset), dataset[0][0]

(60000, <PIL.Image.Image image mode=L size=28x28>)

In [4]:
imgs = [dataset[0][0], dataset[1][0]]
imgs[0]

In [5]:
imgs[1]

In [6]:
len(dataset)

60000

## Investigate DINO V2 from Hugging face

In [14]:
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')
model.cuda()
inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
outputs = model(**inputs)
last_hidden_states = outputs[0]

In [15]:
outputs.last_hidden_state.shape, last_hidden_states.shape

(torch.Size([2, 257, 384]), torch.Size([2, 257, 384]))

In [10]:
inputs = processor(images=imgs, return_tensors="pt")

In [11]:
inputs["pixel_values"].shape

torch.Size([2, 3, 224, 224])

### Batch class encoding using DINO v2

In [20]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)
batch_size = 128
n = len(dataset)
n_batch = (n + batch_size - 1) // batch_size
embedding_list = []
for batch_i in tqdm(range(n_batch)):
    imgs = [dataset[i_][0] for i_ in range(batch_i * batch_size, (batch_i+1) * batch_size) if i_ < n]
    inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list.append(last_hidden_states[:, 0, :].cpu()) # extract cls token embedding 
torch.save(torch.concat(embedding_list), "../datasets/mnst_train_dinov2_small.pt")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [06:49<00:00,  1.15it/s]


In [24]:
dataset = datasets.MNIST('../datasets',
                      train=False,
                      download=True,
                      transform=None)
batch_size = 128
n = len(dataset)
n_batch = (n + batch_size - 1) // batch_size
embedding_list = []
for batch_i in tqdm(range(n_batch)):
    imgs = [dataset[i_][0] for i_ in range(batch_i * batch_size, (batch_i+1) * batch_size) if i_ < n]
    inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list.append(last_hidden_states[:, 0, :].cpu()) # extract cls token embedding 
torch.save(torch.concat(embedding_list), "../datasets/mnst_test_dinov2_small.pt")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:06<00:00,  1.19it/s]


In [14]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)
labels = [dataset[i_][1] for i_ in range(len(dataset))]
torch.save(torch.tensor(labels), "../datasets/mnst_train_labels.pt")

In [15]:
dataset = datasets.MNIST('../datasets',
                      train=False,
                      download=True,
                      transform=None)
labels = [dataset[i_][1] for i_ in range(len(dataset))]
torch.save(torch.tensor(labels), "../datasets/mnst_test_labels.pt")

In [68]:
train_embedding = torch.load("../datasets/mnst_train_dinov2_small.pt")
train_labels = torch.load("../datasets/mnst_train_labels.pt")

## Simulate real bag dataformat with MNST

In [4]:
## Simulate the bag data with encoding only 
## For each data sample, it has random variable bag length, and the data contents should be array with shape bag_length x emb_size
## for each image, there is one emb_size vector encoding. The bag_length, and target information can be stored in the filename. 
## The data sample is saved as a numpy file: bagLength_15_target_1.npy, and the embedding tensors are saved in the file. 
import numpy as np
import torch 
import os
from tqdm import tqdm
import pandas as pd
import hashlib
## This folder is best on SSD as the training will randomly seed the files on the disk
train_folder = "../datasets/MNST_train"
test_folder = "../datasets/MNST_test"


train_embedding = torch.load("../datasets/mnst_train_dinov2_small.pt")
train_labels = torch.load("../datasets/mnst_train_labels.pt")

In [5]:
# Simulate bag length as poisson
train_embedding = torch.load("../datasets/mnst_train_dinov2_small.pt")
train_labels = torch.load("../datasets/mnst_train_labels.pt")
random_seed = 0
np.random.seed(random_seed)
n_total_train = len(train_embedding)
target_label = 9
poisson_length = 25
max_length = 40
n_train = 1000000
labels = []
for i in tqdm(range(n_train)):
    hashFolder1 = int(hashlib.md5(str(i).encode()).hexdigest(), 16) % 100
    hashFolder2 = int(hashlib.md5((str(i) + "f").encode()).hexdigest(), 16) % 100
    folder = f"{train_folder}/{hashFolder1}/{hashFolder2}/"
    if os.path.exists(folder) is False:
        os.makedirs(folder, exist_ok=True)
    bag_length = np.clip(np.random.poisson(poisson_length), 1, max_length)
    random_indices = np.random.randint(n_total_train, size=(bag_length))
    current_array = np.array(train_embedding[random_indices])
    current_label = train_labels[random_indices]
    current_label = int(torch.sum(current_label == 9) >= 4)
    labels.append(current_label)
    filename = "bagLength_%03d_target_%01d_train_%07d"%(bag_length, current_label, i)
    filepath = os.path.join(folder, filename)
    np.save(filepath, current_array)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000000/1000000 [06:38<00:00, 2508.59it/s]


In [6]:
# Simulate bag length as poisson
test_embedding = torch.load("../datasets/mnst_test_dinov2_small.pt")
test_labels = torch.load("../datasets/mnst_test_labels.pt")
random_seed = 2
np.random.seed(random_seed)
n_total_test = len(test_embedding)
target_label = 9
poisson_length = 25
max_length = 40
n_test = 300000
labels = []
for i in tqdm(range(n_test)):
    hashFolder1 = int(hashlib.md5(str(i).encode()).hexdigest(), 16) % 100
    hashFolder2 = int(hashlib.md5((str(i) + "f").encode()).hexdigest(), 16) % 100
    folder = f"{test_folder}/{hashFolder1}/{hashFolder2}/"
    if os.path.exists(folder) is False:
        os.makedirs(folder, exist_ok=True)
    bag_length = np.clip(np.random.poisson(poisson_length), 1, max_length)
    random_indices = np.random.randint(n_total_test, size=(bag_length))
    current_array = np.array(test_embedding[random_indices])
    current_label = test_labels[random_indices]
    current_label = int(torch.sum(current_label == 9) >= 4)
    labels.append(current_label)
    filename = "bagLength_%03d_target_%01d_test_%07d"%(bag_length, current_label, i)
    filepath = os.path.join(folder, filename)
    np.save(filepath, current_array)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300000/300000 [01:44<00:00, 2857.27it/s]


In [8]:
train_files = []
train_folder = "../datasets/MNST_train"
for hashFolder1 in range(100):
    for hashFolder2 in range(100):
        folder = f"{train_folder}/{hashFolder1}/{hashFolder2}/"
        train_files.extend([os.path.join(folder, f_) for f_ in os.listdir(folder) if "train" in f_])
train_df = pd.DataFrame.from_dict({"array_path":train_files})
train_df["bag_length"] = train_df["array_path"].apply(lambda x: int(x.split("/")[-1].split("_")[1]))
train_df["target"] = train_df["array_path"].apply(lambda x: int(x.split("/")[-1].split("_")[3]))
train_df["array_path"] = train_df["array_path"].apply(lambda x: x.strip("../"))
train_df.to_csv("../datasets/mnst_train.csv", index=False)

In [9]:
test_files = []
test_folder = "../datasets/MNST_test"
for hashFolder1 in range(100):
    for hashFolder2 in range(100):
        folder = f"{test_folder}/{hashFolder1}/{hashFolder2}/"
        test_files.extend([os.path.join(folder, f_) for f_ in os.listdir(folder) if "test" in f_])

test_df = pd.DataFrame.from_dict({"array_path":test_files})
test_df["bag_length"] = test_df["array_path"].apply(lambda x: int(x.split("/")[-1].split("_")[1]))
test_df["target"] = test_df["array_path"].apply(lambda x: int(x.split("/")[-1].split("_")[3]))
test_df["array_path"] = test_df["array_path"].apply(lambda x: x.strip("../"))
test_df.to_csv("../datasets/mnst_test.csv", index=False)